## 1 Etude du problème d’optimisation



**1.** 

Sur un intervalle de temps $[t_i;t_{i+1}]$ le tarif d'électricité est supposé constant de valeur $c_i$. Ainsi l'énergie consommée sur cet intervalle de temps étant $(t_{i+1} - t_i)P_i = \Delta t P_i$, le prix a payer est alors $c_i \Delta t P_i$.

Au final, la fatcure totale est obtenue en sommant les factures sur les intervalles $[t_i;t_{i+1}]$ pour $i \in [0,n]$, donc **le prix total à payer est**: 

$$\sum_{i=0}^{n}{c_i \Delta t P_i}=\Delta t  \sum_{i=0}^{n}{c_i P_i}$$

**2.**

**Interpréter l’équation (2). Quels échanges thermiques incorpore-t-elle ? Cette modélisation vous semblet-elle raisonnable ?**

**3.**

L'objectif du problème est de minimiser la facture de chaufage à payer en respectant différentes contraintes.

Ainsi on pose $f(x)=\Delta t  \sum_{i=0}^{n}{c_i P_i}$ afin de chercher à résoudre $\min\limits_{x \in R}{f(x)}$.

On traduit la contrainte de confort en posant **$c_{in}(x)= \begin{pmatrix} T_m - T_i \\  T_i - T_M \\ P_i - P_M \end{pmatrix}  \le 0$**.